# Reporte CRM 
8/02/23

In [4]:
from numpy import select
import pandas as pd
import re

import cx_Oracle as oracle
import win32com.client

import sys
from datetime import *
from datetime import date
from datetime import timedelta

import tkinter as tk
from tkinter import filedialog
from tkinter.messagebox import showinfo
from tkinter import simpledialog

import warnings
warnings.filterwarnings('ignore')

# Conexión a base de datos Oracle
dsn_tns = oracle.makedsn('10.60.4.165', '1530', service_name='OBI500')
conn = oracle.connect(user='CONS_MANUFACTURA', password='Orbis2021*', dsn=dsn_tns)
c = conn.cursor()

query="""SELECT NUMERO_SOLICITUD,
            PROPIETARIO_INCIDENTE,
            NOMBRE_CLIENTE,
            ESTADO_SOLICITUD,
            EXTERNAL_ATTRIBUTE_6 as DEFECTO_INICIAL,
            FECHA_CREACION_SOLICITUD,
            FECHA_SOLUCION,
            EXTERNAL_ATTRIBUTE_3 as FECHA_RECOGIDA,
            
            TIPO_SOLICITUD,
            SUBTIPO_SOLICITUD
       FROM ods_pdn.ODS_CRM_Quimicos 
       WHERE (TIPO_SOLICITUD LIKE 'ADC - RECLAMO%' OR TIPO_SOLICITUD = 'ADC - QUEJA') AND  
            ESTADO_SOLICITUD NOT LIKE 'CANCELADO' AND
            FECHA_CREACION_SOLICITUD >= TO_DATE('2021-01-01','RRRR-MM-DD') 
       ORDER BY PROPIETARIO_INCIDENTE, NUMERO_SOLICITUD"""

quejas_reclamos = pd.read_sql(con=conn,sql=query)

segmentacion = pd.read_excel(r'segmentacion.xlsx')
quejas_reclamos = quejas_reclamos.merge(segmentacion, how= 'left', left_on='NOMBRE_CLIENTE', right_on='Cliente')

#quejas_reclamos['ESTADO_SOLICITUD'] = quejas_reclamos['ESTADO_SOLICITUD'].str.strip('.')

In [5]:
query = "SELECT * FROM ods_pdn.ODS_CRM_Quimicos where ROWNUM<2"
colnames = pd.read_sql(con=conn,sql=query)
colnames.columns

Index(['NEGOCIO', 'CLASIFICACION', 'CANAL', 'NOMBRE_CLIENTE', 'NUMERO_CUENTA',
       'DOMICILIO_CLIENTE', 'DEPARTAMENTO_CLIENTE', 'CIUDAD_CLIENTE',
       'PAIS_CLIENTE', 'REGIONAL', 'NOMBRE_CONTACTO', 'TEL_CONTACTO',
       'EMAIL_CONTACTO', 'NUMERO_SOLICITUD', 'TIPO_SOLICITUD',
       'SUBTIPO_SOLICITUD', 'RESUMEN_PROBLEMA', 'RESUMEN_SOLICITUD',
       'CODIGO_SOLUCION', 'ARTICULO', 'ESTADO_SOLICITUD',
       'FECHA_CREACION_SOLICITUD', 'FECHA_CIERRE_SOLICITUD',
       'FECHA_REQ_RESPUESTA', 'FECHA_RESPUESTA', 'FECHA_REQ_SOLUCION',
       'FECHA_SOLUCION', 'SOLICITUD_CREADA_POR', 'SOLICITUD_MODIFICADA_POR',
       'FECHA_ULTIMA_MODIF', 'PROPIETARIO_INCIDENTE', 'GRUPO_PROPIETARIO',
       'NUMERO_TAREA_SOLICITUD', 'TIPO_TAREA_SOLICITUD',
       'TEMA_TAREA_SOLICITUD', 'ESTADO_TAREA_SOLICITUD',
       'PRIORIDAD_TAREA_SOLICITUD', 'F_PLANEADA_INICIO_TAREA',
       'F_PLANEADA_FIN_TAREA', 'F_PROGRAMADA_INICIO_TAREA',
       'F_PROGRAMADA_FIN_TAREA', 'F_REAL_INICIO_TAREA', 'F_REAL_FIN_TA

In [6]:
quejas_reclamos['ESTADO_SOLICITUD'].unique()

array(['CERRADO ADC', 'MATERIAL RECOGIDO', 'ABIERTO ADC',
       'MATERIAL POR RECOGER', 'ANALIZADO'], dtype=object)

In [7]:
quejas_reclamos_cerrados = quejas_reclamos[quejas_reclamos.ESTADO_SOLICITUD == "CERRADO ADC"]

quejas_reclamos_abiertos = quejas_reclamos[quejas_reclamos.ESTADO_SOLICITUD != "CERRADO ADC"]
quejas_reclamos_abiertos['FECHA_RECOGIDA'] = pd.to_datetime(quejas_reclamos_abiertos['FECHA_RECOGIDA'])



In [8]:
quejas_reclamos_cerrados.ESTADO_SOLICITUD.unique()

array(['CERRADO ADC'], dtype=object)

## Quejas

In [9]:
quejas = quejas_reclamos_abiertos[quejas_reclamos_abiertos.TIPO_SOLICITUD == "ADC - QUEJA"]
quejas['FECHA ESPERADA CIERRE'] = quejas['FECHA_CREACION_SOLICITUD'] + timedelta(days=12)
quejas['Días para Cierre'] = (quejas['FECHA ESPERADA CIERRE'] - date.today()).days


for propietario in quejas['PROPIETARIO_INCIDENTE'].unique():

    if propietario == None:
        continue

 #def hacer_html_quejas(df_quejas, propietario)
    #casos = df_quejas[df_quejas['PROPIETARIO_INCIDENTE']==propietario]
    
    casos = quejas[quejas['PROPIETARIO_INCIDENTE']==propietario]
    casos.sort_values(by = 'FECHA ESPERADA CIERRE', ascending=False, inplace=True)
    
    dias_para_cierre = int( (casos['FECHA ESPERADA CIERRE'].tolist()[-1] - date.today()).days )
    nombre = casos['PROPIETARIO_INCIDENTE'].tolist()[0].rsplit(',', 1)[1].strip().title()
    
    #ASUNTO

    if casos.shape[0] == 1:

        if dias_para_cierre < 0: 
            asunto = f"¡{nombre} tienes un CRM asignado, atrasado por {-dias_para_cierre} días!"
        elif dias_para_cierre == 0: 
            asunto = f"¡{nombre} tienes un CRM asignado, debes solucionarlo hoy!"
        else: 
            asunto = f"¡{nombre} tienes un CRM asignado, tienes {dias_para_cierre} días para solucionarlo!"

    else:

        if dias_para_cierre < 0: 
            asunto = f"¡{nombre} tienes CRMs asignados, atrasados hasta por {-dias_para_cierre} días!"
        elif dias_para_cierre == 0: 
            asunto = f"¡{nombre} tienes CRMs asignados, algunos debes solucionarlos hoy!"
        else: 
            asunto = f"¡{nombre} tienes CRMs asignados, tienes {dias_para_cierre} días para solucionarlos!"

    # CUERPO

    if casos.shape[0] == 1:

        body = f"""\
        <html>
        <head></head>
        <body>
            Buenos días {nombre},<br> Te recordamos que tienes un caso abierto en la plataforma del CRM.<br><br>
            Este es el detalle de tu caso:</p>"""

    else:

        body = f"""\
        <html>
        <head></head>
        <body>
            Buenos días {nombre},<br> Te recordamos que tienes varios casos abiertos en la plataforma del CRM.<br><br>
            Estos son los detalles de tus casos:</p>"""

    
    for caso in casos.index:

        fila = casos[casos.index == caso]
        dias_para_cierre_caso = int( (fila['FECHA ESPERADA CIERRE'].tolist()[0] - date.today()).days )
        fecha_cierre = fila['FECHA ESPERADA CIERRE'].tolist()[0].strftime("%d/%m/%Y")

        if (dias_para_cierre_caso < 0):
            parte_uno = f"<b>Este caso está atrasado en tu área por {-dias_para_cierre_caso} días, deberíamos haber dado respuesta al cliente el {fecha_cierre}</b><br><br>"

        elif (dias_para_cierre_caso == 0):
            parte_uno = f"<b>Debes solucionar este caso hoy, el cliente está esperando nuestra respuesta</b><br><br>"
        
        elif (dias_para_cierre_caso > 0):
            parte_uno = f"<b>Tienes {dias_para_cierre_caso} días para solucionar este caso, tenemos el compromiso de dar respuesta al cliente el {fecha_cierre}"
        
        tabla = fila[['NUMERO_SOLICITUD','Cliente','Segmento','DEFECTO_INICIAL','FECHA_CREACION_SOLICITUD','Días para Cierre']]

        tabla.rename(columns = {
                    'NUMERO_SOLICITUD': 'Número Solicitud',
                    'DEFECTO_INICIAL': 'Defecto Inicial',
                    'FECHA_CREACION_SOLICITUD': 'Fecha de Creación'
                        }, inplace = True)

        body = body + f"{parte_uno}<br>{tabla.to_html(index=False)}</p>"
    
    body = body + "Gracias por tu compromiso, con tu gestión lograremos dar respuesta al cliente</p>Saludos"""

    print(propietario, body)
    #return asunto, body


TypeError: unsupported operand type(s) for -: 'DatetimeArray' and 'datetime.date'

## Reclamos Abiertos

### Caso 1
Para:
Calidad y Desempeño, 
Cadena de Suministro, 
Otros Errores

- Fecha inicio
- Fecha de Solución max 12 dias corridos (Análisis)
- Fecha Recogida max 8 Días corridos (Logistica)
- Fecha de Cierre max 2 días corridos (Nota Crédito)

Total: 22 Días Corridos

In [10]:
subtipos_1 = ['ADC - CALIDAD/DESEMPENO', 'ADC - OTROS ERRORES', 'ADC - CADENA DE SUMINISTRO']
subtipos_2 = ['ADC - DEVOLUCIONES INMEDIATAS (LOGISTICA)','ADC - ERRORES HUMANOS']

In [11]:
reclamos = quejas_reclamos_abiertos[quejas_reclamos_abiertos.TIPO_SOLICITUD != "ADC - QUEJA"]

caso1 = reclamos[reclamos.SUBTIPO_SOLICITUD.apply(lambda x: x in subtipos_1)]
caso2 = reclamos[reclamos.SUBTIPO_SOLICITUD.apply(lambda x: x in subtipos_2)]


In [12]:
# PARA EL CASO 1
caso1['FECHA ESPERADA CIERRE IDEAL'] = caso1['FECHA_CREACION_SOLICITUD'] + timedelta(days=22)
caso1['FECHA ESPERADA SOLUCION'] = caso1['FECHA_CREACION_SOLICITUD'] + timedelta(days=12)
caso1['FECHA ESPERADA RECOGIDA'] = caso1['FECHA_SOLUCION'] + timedelta(days=8)
caso1['FECHA ESPERADA CIERRE'] = caso1['FECHA_RECOGIDA'] + timedelta(days=2)

# PARA EL CASO 2
caso2['FECHA ESPERADA CIERRE IDEAL'] = caso2['FECHA_CREACION_SOLICITUD'] + timedelta(days=12)
caso2['FECHA ESPERADA SOLUCION'] = caso2['FECHA_CREACION_SOLICITUD'] + timedelta(days=2)
caso2['FECHA ESPERADA RECOGIDA'] = caso2['FECHA_SOLUCION'] + timedelta(days=8)
caso2['FECHA ESPERADA CIERRE'] = caso2['FECHA_RECOGIDA'] + timedelta(days=2)


In [13]:
caso2

,NUMERO_SOLICITUD,PROPIETARIO_INCIDENTE,NOMBRE_CLIENTE,ESTADO_SOLICITUD,DEFECTO_INICIAL,FECHA_CREACION_SOLICITUD,FECHA_SOLUCION,FECHA_RECOGIDA,TIPO_SOLICITUD,SUBTIPO_SOLICITUD,Cliente,Segmento,FECHA ESPERADA CIERRE IDEAL,FECHA ESPERADA SOLUCION,FECHA ESPERADA RECOGIDA,FECHA ESPERADA CIERRE
145,4580934,"ARANGO. URREGO, MANUEL",ROCSA COLOMBIA SAS,MATERIAL RECOGIDO,Averia tte/Defecto Material empaque/mcia trocada,2023-02-08 09:58:40,2023-02-08 10:03:06,2023-01-27,ADC - RECLAMO (NUEVO),ADC - DEVOLUCIONES INMEDIATAS (LOGISTICA),NaN,NaN,2023-02-20 09:58:40,2023-02-10 09:58:40,2023-02-16 10:03:06,2023-01-29
191,4585648,"CUSSO. MORENO, JUAN JOSE",NOVASUIN S.A.S,ABIERTO ADC,ERROR EN EL PRECIO,2023-02-27 18:07:11,NaT,2023-02-27,ADC - RECLAMO (NUEVO),ADC - ERRORES HUMANOS,NOVASUIN S.A.S,Dinámico,2023-03-11 18:07:11,2023-03-01 18:07:11,NaT,2023-03-01
545,4577332,"SIERRA. TORRES, EVERING EUSEBIO",LA TEJADA S.A.S,ABIERTO ADC,Averia tte/Defecto Material empaque/mcia trocada,2023-01-26 12:00:08,NaT,NaT,ADC - RECLAMO (NUEVO),ADC - DEVOLUCIONES INMEDIATAS (LOGISTICA),LA TEJADA S.A.S,Esencial,2023-02-07 12:00:08,2023-01-28 12:00:08,NaT,NaT


In [14]:
caso1

,NUMERO_SOLICITUD,PROPIETARIO_INCIDENTE,NOMBRE_CLIENTE,ESTADO_SOLICITUD,DEFECTO_INICIAL,FECHA_CREACION_SOLICITUD,FECHA_SOLUCION,FECHA_RECOGIDA,TIPO_SOLICITUD,SUBTIPO_SOLICITUD,Cliente,Segmento,FECHA ESPERADA CIERRE IDEAL,FECHA ESPERADA SOLUCION,FECHA ESPERADA RECOGIDA,FECHA ESPERADA CIERRE
189,4569202,"CUSSO. MORENO, JUAN JOSE",NOVASUIN S.A.S,MATERIAL RECOGIDO,APARIENCIA GELES,2022-12-20 15:38:00,2023-01-13 12:40:36,NaT,ADC - RECLAMO (NUEVO),ADC - CALIDAD/DESEMPENO,NOVASUIN S.A.S,Dinámico,2023-01-11 15:38:00,2023-01-01 15:38:00,2023-01-21 12:40:36,NaT
270,4575180,"GOMEZ. RAMIREZ, DIANA ASTRID",QUIMTIA S.A.,MATERIAL POR RECOGER,APARIENCIA CONTAMINACIN,2023-01-18 18:49:11,2023-02-02 14:10:26,1900-01-01,ADC - RECLAMO (NUEVO),ADC - CALIDAD/DESEMPENO,QUIMTIA S.A.,Dinámico,2023-02-09 18:49:11,2023-01-30 18:49:11,2023-02-10 14:10:26,1900-01-03
318,4583222,"JARAMILLO. OSORIO, NELCY JANETH",PINTUEXITO SAS,MATERIAL POR RECOGER,Otros errores Cadena,2023-02-17 13:54:09,2023-02-21 17:46:52,NaT,ADC - RECLAMO (NUEVO),ADC - CADENA DE SUMINISTRO,PINTUEXITO SAS,Esencial,2023-03-11 13:54:09,2023-03-01 13:54:09,2023-03-01 17:46:52,NaT
321,4579674,"LEON. BOLIVAR, YEXANDA DEL VALLE",OXIGENADOS Y DERIVADOS S.A.S.,ABIERTO ADC,Errores en Peso,2023-02-03 15:57:35,NaT,NaT,ADC - RECLAMO (NUEVO),ADC - OTROS ERRORES,OXIGENADOS Y DERIVADOS S.A.S.,Esencial,2023-02-25 15:57:35,2023-02-15 15:57:35,NaT,NaT
322,4582492,"LEON. BOLIVAR, YEXANDA DEL VALLE",OXIGENADOS Y DERIVADOS S.A.S.,ABIERTO ADC,Errores en Peso,2023-02-14 15:24:48,NaT,NaT,ADC - RECLAMO (NUEVO),ADC - OTROS ERRORES,OXIGENADOS Y DERIVADOS S.A.S.,Esencial,2023-03-08 15:24:48,2023-02-26 15:24:48,NaT,NaT
323,4582846,"LEON. BOLIVAR, YEXANDA DEL VALLE",PINTURAS PRIME S.A.,ABIERTO ADC,Errores en Peso,2023-02-16 10:04:31,NaT,2000-01-01,ADC - RECLAMO (NUEVO),ADC - OTROS ERRORES,PINTURAS PRIME S.A.,Dinámico,2023-03-10 10:04:31,2023-02-28 10:04:31,NaT,2000-01-03
356,4564378,"OSORIO. HENAO, HADERSON ALEXANDER",PINTUCO COLOMBIA S.A.S.,ABIERTO ADC,ESTABILIDAD EN ALMACENAMIENTO,2022-12-07 10:31:05,NaT,NaT,ADC - RECLAMO (NUEVO),ADC - CALIDAD/DESEMPENO,PINTUCO COLOMBIA S.A.S.,Dinámico,2022-12-29 10:31:05,2022-12-19 10:31:05,NaT,NaT
357,4577124,"OSORIO. HENAO, HADERSON ALEXANDER",PINTUCO COLOMBIA S.A.S.,ABIERTO ADC,CONTAMINACION,2023-01-25 15:48:54,NaT,NaT,ADC - RECLAMO (NUEVO),ADC - CALIDAD/DESEMPENO,PINTUCO COLOMBIA S.A.S.,Dinámico,2023-02-16 15:48:54,2023-02-06 15:48:54,NaT,NaT
434,4575650,"RAMIREZ. ORTIZ, ADRIAN MAURICIO",UNION COMERCIAL ROPTIE S.A. - UNICOR S.A.,ABIERTO ADC,CHORREO,2023-01-20 11:26:52,NaT,NaT,ADC - RECLAMO (NUEVO),ADC - CALIDAD/DESEMPENO,UNION COMERCIAL ROPTIE S.A. - UNICOR S.A.,Cumbre,2023-02-11 11:26:52,2023-02-01 11:26:52,NaT,NaT
436,4576798,"RENDON. ROBLEDO, JUAN JOSE",C.A. QUIMICA INTEGRADA INTEQUIM,ABIERTO ADC,OJO DE PEZ,2023-01-24 17:25:23,NaT,NaT,ADC - RECLAMO (NUEVO),ADC - CALIDAD/DESEMPENO,C.A. QUIMICA INTEGRADA INTEQUIM,Cumbre,2023-02-15 17:25:23,2023-02-05 17:25:23,NaT,NaT


In [7]:
# ESTO SI YA SIRVE PARA AMBOS CASOS

caso1.reset_index(inplace=True, drop=True)

# DIAS PARA SOLUCION EN EL AREA EN CUESTION, ASUME QUE LA PERSONA ES LA INDICADA
for i in range(caso1.shape[0]):

    if caso1['ESTADO_SOLICITUD'].iloc[i] == 'ABIERTO ADC' or \
       caso1['ESTADO_SOLICITUD'].iloc[i] == 'POR SOLUCION ADC':             # Si no lo han solucionado, está en el primer paso
        
        # Tienes x dias para solucionar, primer dueño. Si da negativo, esta atrasado en esta área.
        caso1.loc[i,'Días para Solución'] = (caso1['FECHA ESPERADA SOLUCION'].iloc[i].date() - date.today()).days
        caso1.loc[i,'Columna Fecha a Reportar'] = 'FECHA ESPERADA SOLUCION'

    elif caso1['ESTADO_SOLICITUD'].iloc[i] != 'ABIERTO ADC' and \
         caso1['ESTADO_SOLICITUD'].iloc[i] != 'POR SOLUCION ADC':           # Si ya lo solucionaron en el primer paso
       
            # Aqui debería chequear que no este ya recogido, pero la fecha de recogida no esta confiable y no hay estado que diga tampoco
            # if caso1['FECHA_RECOGIDA'] == NaT:                            # y no tiene fecha de recogida, está en logistica

        # Tienes x dias para solucionar, logistica. Si da negativo, esta atrasado en esta área.
        caso1.loc[i,'Días para Solución'] = (caso1['FECHA ESPERADA RECOGIDA'].iloc[i].date() - date.today()).days
        caso1.loc[i,'Columna Fecha a Reportar'] = 'FECHA ESPERADA RECOGIDA'

            #elif caso1['FECHA_RECOGIDA'] != NaT:                           # Si ya tiene fecha de recogida, está en SAC

            # Tienes x dias para solucionar, SAC. Si da negativo, esta atrasado en esta área. Nunca deberia ser mayor a 2.
            #caso1.loc[i,'Días para Solución'] = (caso1['FECHA ESPERADA CIERRE'].iloc[i].date() - date.today()).days
            #caso1.loc[i,'Columna Fecha a Reportar'] = 'FECHA ESPERADA CIERRE'
        
        
    # Encima, cuanto lleva abierto. Si da mas de 22 está retrasado pa todo el mundo.
    caso1.loc[i,'Dias Desde Creacion'] = (date.today()-caso1['FECHA_CREACION_SOLICITUD'].iloc[i].date()).days

    # Recuerda que igual deberia cerrar el FECHA ESPERADA CIERRE IDEAL en ____ días, si da negativo está atrasado
    caso1.loc[i,'Dias para Cierre'] = (caso1['FECHA ESPERADA CIERRE IDEAL'].iloc[i].date() - date.today()).days  

In [15]:
# GENERAR EL TEXTO DEL CORREO

for propietario in caso1['PROPIETARIO_INCIDENTE'].unique():

    if propietario == None:
        continue

    casos = caso1[caso1['PROPIETARIO_INCIDENTE']==propietario]
    
    casos.sort_values(by = 'Días para Solución', ascending=False, inplace=True)
    
    dias_para_sln = int(casos['Días para Solución'].tolist()[-1])
    nombre = casos['PROPIETARIO_INCIDENTE'].tolist()[0].rsplit(',', 1)[1].strip().title()

    #ASUNTO

    if casos.shape[0] == 1:

        if dias_para_sln < 0: 
            asunto = f"¡{nombre} tienes un CRM asignado, atrasado por {-dias_para_sln} días!"
        elif dias_para_sln == 0: 
            asunto = f"¡{nombre} tienes un CRM asignado, debes solucionarlo hoy!"
        else: 
            asunto = f"¡{nombre} tienes un CRM asignado, tienes {dias_para_sln} días para solucionarlo!"

    else:

        if dias_para_sln < 0: 
            asunto = f"¡{nombre} tienes CRMs asignados, atrasados hasta por {-dias_para_sln} días!"
        elif dias_para_sln == 0: 
            asunto = f"¡{nombre} tienes CRMs asignados, algunos debes solucionarlos hoy!"
        else: 
            asunto = f"¡{nombre} tienes CRMs asignados, tienes {dias_para_sln} días para solucionarlos!"

    # CUERPO

    header = '...'

    body = f"""\
    <!DOCTYPE html>
    <html>

    <head>  
            <link href='https://fonts.googleapis.com/css2?family=Nanum+Gothic&display=swap' rel='stylesheet' type='text/css'>
            <link href='https://fonts.googleapis.com/css2?family=Lobster&display=swap' rel='stylesheet' type='text/css'>
        <style type="text/css">
            span {{
                font-style: oblique;
                font-family: 'Nanum Gothic', sans-serif;
                font-size: 1.5em;
                color: #0E0E0E;
                }}

            .dataframe {{
                table-layout: fixed;
                width: 85%;
                height: 35vh;
                border: 0;
                margin-left: auto;
                margin-right: auto;
            }}

            tbody {{
                text-align: center;
                padding: .5em;
                font-size: 1.2em;
                font-weight: bold;
                font-family: 'Nanum Gothic', sans-serif;
                color: #185192;
            }}

            thead {{
                background: #84b343;
                font-weight: bold;
                font-size: 1.3em;
                color: white;
                border: 0;
            }}

            thead th,
            tbody td,
            tbody th {{
                border:0;
                border-radius: .4em;
            }}

        </style>
    </head>    
    
        <body style="background-image: url('/fondo.jpg'); background-repeat: no-repeat">
            <div>
                <center> <img alt="" src="{header}" width="500" height="179"/> </center>
                <p style="font-family: Helvetica; text-align: center;">
                    <span style="font-family: 'Nanum Gothic', Helvetica, Arial, sans-serif !important;">"""

    if casos.shape[0] == 1:

        body = body + f"""¡Buenos días {nombre}!<br> Te recordamos que tienes un caso abierto en la plataforma del CRM.<br><br>
            Este es el detalle de tu caso:</p></span>"""

    else:

        body = body + f"""¡Buenos días {nombre}!<br> te recordamos que tienes varios casos abiertos en la plataforma del CRM.<br><br>
            Estos son los detalles de tus casos:</p></span>"""

    for caso in casos.index:

        fila = casos[casos.index == caso]
        dias_para_sln_caso = int(fila['Días para Solución'].tolist()[0])
        dias_para_cierre_caso = int(fila['Dias para Cierre'].tolist()[0])
        fecha_cierre = fila['FECHA ESPERADA CIERRE IDEAL'].tolist()[0].strftime("%d/%m/%Y")

        if (dias_para_sln_caso < 0) & (dias_para_cierre_caso < 0):
            parte_uno = f"<span><b>Este caso está atrasado en tu área por {-dias_para_sln_caso} días, "
            parte_dos = f"""y deberíamos haber dado respuesta al cliente el {fecha_cierre},  
                            hace {-dias_para_cierre_caso} días</b></span><br><br>"""

        elif (dias_para_sln_caso < 0) & (dias_para_cierre_caso == 0):
            parte_uno = f"<span><b>Este caso está atrasado en tu área por {-dias_para_sln_caso} días, "
            parte_dos = f"y el cliente está esperando nuestra respuesta HOY</b></span><br><br>"
        
        elif (dias_para_sln_caso < 0) & (dias_para_cierre_caso > 0):
            parte_uno = f"<span><b>Este caso está atrasado en tu área por {-dias_para_sln_caso} días, "
            parte_dos = f"""y tenemos el compromiso de dar respuesta al cliente el {fecha_cierre}, 
                            en {dias_para_cierre_caso} días</b></span><br><br>"""

        elif (dias_para_sln_caso == 0) & (dias_para_cierre_caso < 0):
            parte_uno = f"<span><b>Debes solucionar este caso hoy en tu área, "
            parte_dos = f"""y deberíamos haber dado respuesta al cliente el {fecha_cierre},  
                            hace {-dias_para_cierre_caso} días</b></span><br><br>"""

        elif (dias_para_sln_caso == 0) & (dias_para_cierre_caso == 0):
            parte_uno = f"<span><b>Debes solucionar este caso hoy en tu área, "
            parte_dos = f"y el cliente está esperando nuestra respuesta HOY</b></span><br><br>"
        
        elif (dias_para_sln_caso == 0) & (dias_para_cierre_caso > 0):
            parte_uno = f"<span><b>Debes solucionar este caso hoy en tu área, "
            parte_dos = f"""y tenemos el compromiso de dar respuesta al cliente el {fecha_cierre}, 
                            en {dias_para_cierre_caso} días</b></span><br><br>"""

        elif (dias_para_sln_caso > 0) & (dias_para_cierre_caso < 0):
            parte_uno = f"<span><b>Tienes {dias_para_sln_caso} días para solucionar este caso en tu área, "
            parte_dos = f"""pero por favor recuerda que deberíamos haber dado respuesta al cliente el {fecha_cierre}, 
                            hace {-dias_para_cierre_caso} días</b></span><br><br>"""

        elif (dias_para_sln_caso > 0) & (dias_para_cierre_caso == 0):
            parte_uno = f"<span><b>Tienes {dias_para_sln_caso} días para solucionar este caso en tu área, "
            parte_dos = f"pero por favor recuerda que el cliente está esperando nuestra respuesta HOY</b></span><br><br>"
        
        elif (dias_para_sln_caso > 0) & (dias_para_cierre_caso > 0):
            parte_uno = f"<span><b>Tienes {dias_para_sln_caso} días para solucionar este caso en tu área, "
            parte_dos = f"""pero por favor recuerda que tenemos el compromiso de dar respuesta al cliente el {fecha_cierre}, 
                            en {dias_para_cierre_caso} días</b></span><br><br>"""

        tabla = fila[['NUMERO_SOLICITUD', 'ESTADO_SOLICITUD','Cliente','Segmento','DEFECTO_INICIAL','FECHA_CREACION_SOLICITUD','Días para Solución',fila['Columna Fecha a Reportar'].tolist()[0]]]
        tabla.rename(columns = {
                    'NUMERO_SOLICITUD': 'Número Solicitud',
                    'DEFECTO_INICIAL': 'Defecto Inicial',
                    'FECHA_CREACION_SOLICITUD': 'Fecha de Creación',
                    fila['Columna Fecha a Reportar'].tolist()[0]: fila['Columna Fecha a Reportar'].tolist()[0].title()
                        }, inplace = True)

        body = body + f"""<center>{parte_uno}{parte_dos}<br>
                       <div style="padding:10px 10px 500px 500px; display: flex; align-items: center; justify-content: center;">
                        {tabla.to_html(
                            index=False,
                            classes=['dataframe', 'tbody', 'thead'],
                            border=0,
                            col_space='20vw')}                     
                        <br><br></div></center>"""
    
    body = body + '''
        <p style="font-family: Helvetica; text-align: center;">
            <span>Gracias por tu compromiso, con tu gestión lograremos dar respuesta al cliente<br>
                    Saludos</span></p>
    </div>
    </body>
    </html>
    '''
    
    #if propietario == "RENDON. ROBLEDO, JUAN JOSE":
        #print(body)

    #ENVIO DEL CORREO

    outlook = win32com.client.Dispatch('outlook.application')

    mail = outlook.CreateItem(0)
    mail.To = 'pilar.mejia@andercol.com.co; chelsy.mena@andercol.com.co'
    mail.Subject = asunto
    #mail.display()
    bodystart = re.search("<body.*?>", mail.HTMLBody)
    mail.HTMLBody = re.sub(bodystart.group(), bodystart.group()+ body, mail.HTMLBody)
    mail.CC = 'ana.morales@andercol.com.co'
    mail.Send()

#print('Emails ready')

In [42]:
correos = pd.read_csv('bd_correos.csv', sep = ";")
correos.head()

,NOMBRE,CORREO
0,"AGUDELO. ALVAREZ, VIVIANA MARIA",viviana.agudelo@andercol.com.co
1,"AGUDELO. ARBOLEDA, JUAN FERNANDO",juan.agudeloa@andercol.com.co
2,"AGUDELO. GIRALDO, LEIDY VANESA",leidy.agudelo@andercol.com.co
3,"AGUILAR. COLINA, JONATHAN",jonathan.aguilar@andercol.com.co
4,"AGUILAR. TORREALBA, JOSE MIGUEL",jose.aguilar@andercol.com.co


In [44]:
for propietario in caso2['PROPIETARIO_INCIDENTE'].unique():
    try:
        correo = correos[correos.NOMBRE == propietario].CORREO.tolist()[0]
    except:
        print("Falta el correo de {propietario} en la Base de Datos\nNo se le envió el informe esta vez, por favor agreguelo antes de mañana.")
        continue
    print(propietario, correo)

CUSSO. MORENO, JUAN JOSE juan.cusso@andercol.com.co
SIERRA. TORRES, EVERING EUSEBIO evering.sierra@andercol.com.co
